<a href="https://colab.research.google.com/github/armandossrecife/mysummary/blob/main/text_summary_only_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resumo de texto em português usando apenas o Transformers

Lendo o conteúdo de um arquivo pdf e fazer o resumo desse conteudo em Português.

Os modelos estáveis que usam Transformers só reconhecem texto em inglês

1. Ler o conteúdo em português
2. Traduz o conteúdo de português para inglês (usando o modelo unicamp-dl/translation-pt-en-t5)
3. Faz o resumo do texto em inglês (usando o modelo Falconsai/text_summarization)
4. Traduz o resumo de inglês para português (usando o modelo unicamp-dl/translation-en-pt-t5)
5. Exibe o resultado do resumo em português

## Instalação das dependências

In [1]:
!pip install transformers

In [2]:
!pip install PyPDF2

## Baixa o arquivo que será resumido

In [3]:
!wget https://raw.githubusercontent.com/armandossrecife/mysummary/refs/heads/main/pdfs/d1.pdf

--2024-10-19 16:40:24--  https://raw.githubusercontent.com/armandossrecife/mysummary/refs/heads/main/pdfs/d1.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1850029 (1.8M) [application/octet-stream]
Saving to: ‘d1.pdf’

d1.pdf              100%[===================>]   1.76M  --.-KB/s    in 0.01s   

2024-10-19 16:40:25 (167 MB/s) - ‘d1.pdf’ saved [1850029/1850029]



## Funções de Apoio

In [23]:
import PyPDF2
import os
from datetime import datetime
# Load pipeline from transformers
from transformers import pipeline
from tqdm import tqdm

def extract_pdf_text(pdf_path):
    try:
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)

            pdf_text = {}
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                pdf_text[page_num + 1] = text
            return pdf_text
    except FileNotFoundError:
        print(f"Error: PDF file not found at {pdf_path}")
    except PermissionError:
        print(f"Error: Permission denied to access PDF file at {pdf_path}")
    except Exception as e:
        print(f"Error: An unexpected error occurred: {e}")

## Ler o conteúdo texto do arquivo pdf

In [4]:
texto_documento = ""
meu_conteudo =  extract_pdf_text(pdf_path="d1.pdf")

print(f"Quantidade de páginas: {len(meu_conteudo.keys())}")

for k,v in meu_conteudo.items():
  print(f"Página: {k}")
  print(f"Texto: {v}")
  texto_documento = texto_documento + v

Quantidade de páginas: 25
Página: 1
Texto: AO CHEFE DO DEPARTAMENTO DE COMPUTAÇÃO  DA UNIVERSIDADE FEDERAL DO PIAUÍ  Caro Chefe de Departamento, Eu Armando Soares Sousa, Matrícula UFPI 1886865 lotado no Departamento de Computação do Centro de Ciência da Natureza da Universidade Federal do Piauí solicita sua concordância quanto minha PRORROGAÇÃO DE AFASTAMENTO a fim de dar continuidade ao curso de Pós-Graduação em Ciência da Computação em nível de Doutorado na (Área de Concentração: Ciência da Computação) na Universidade Federal do Ceará, por mais 6 (seis) meses, a partir de 23/02/2022, em conformidade com a Resolução No. 186/06-CEPEX. Além disso, gostaria que o senhor anexasse a meu pedido o documento referente ao plano de qualificação 2022 aprovado na Assembleia Departamental informando a deliberação favorável a meu pedido de afastamento em 2022 para poder concluir minhas atividades do doutorado. Fortaleza, 21, de Dezembro de 2021  __________________________________  (Assinatura) 
Sol

## Traduz do português para o inglês

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [6]:
tokenizer_pt_en = AutoTokenizer.from_pretrained("unicamp-dl/translation-pt-en-t5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
model_pt_en = AutoModelForSeq2SeqLM.from_pretrained("unicamp-dl/translation-pt-en-t5")

In [24]:
len(texto_documento)

31565

In [25]:
pten_pipeline = pipeline('text2text-generation', model=model_pt_en, tokenizer=tokenizer_pt_en)

In [26]:
texto_documento_to_english = ""

with tqdm(total=len(meu_conteudo)) as pbar:
  for key, value in meu_conteudo.items():
    pbar.update(1)
    if len(value) > 1000:
      my_prompt_pt_to_en = f"translate Portuguese to English: {value[:1000]}"
    else:
      my_prompt_pt_to_en = f"translate Portuguese to English: {value}"
    resposta = pten_pipeline(my_prompt_pt_to_en, max_length=1000)
    texto_documento_to_english = texto_documento_to_english + resposta[0]['generated_text']
    tqdm.write(f"Traduzindo a página {key}")

  8%|▊         | 2/25 [01:39<19:02, 49.66s/it]

Traduzindo a página 1


 12%|█▏        | 3/25 [02:07<14:53, 40.60s/it]

Traduzindo a página 2


 16%|█▌        | 4/25 [02:26<11:28, 32.81s/it]

...

Traduzindo a página 23


100%|██████████| 25/25 [18:40<00:00, 53.24s/it]

Traduzindo a página 24


100%|██████████| 25/25 [18:57<00:00, 45.52s/it]

Traduzindo a página 25


In [33]:
print(len(texto_documento_to_english))

27357


In [34]:
print(texto_documento_to_english)

THE CHEFE OF THE DEPARTION OF THE FEDERAL UNIVERSITY OF PIAUÍ Dear Head of Department, Eu Armando Soares Sousa, UFPI Registry 1886865 located in the Department of Computing, Center of Nature Science, Federal University of Piauí, requests your agreement as to my PRORROGATION in order to continue the graduate course in computer science at the Federal University of Ceará, for another 6 months, from February 23, 22, in accordance with Resolution No. 186/06-CEPEX. In addition, I would like you to attach to my request the document relating to the qualification plan 2022 approved in the Department Assembly informing the favourable decision on my request for removal in 2022 to complete my doctoral activities. Fortaleza, 21, December 2021 ____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

## Faz o resumo do texto em inglês

In [35]:
my_summarizer = pipeline("summarization", model="Falconsai/text_summarization")

In [36]:
my_summary_in_english = my_summarizer(texto_documento_to_english, max_length=5000, min_length=30, do_sample=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (8035 > 512). Running this sequence through the model will result in indexing errors


In [37]:
if len(my_summary_in_english) > 0:
  print(my_summary_in_english[0]['summary_text'])

DEVELOPMENT OF THE FEDERAL UNIVERSITY OF PIAU Dear Head of Department, Eu Armando Sousa, UFPI Registry 1886865 located in the Department of Computing, Center of Nature Science, Federal University of Ceará . This certificate aims to comply with the adversities caused by the pandemic of Corona Virus in the years of 2020 and 2021 . Thesis or Dissertation Project Summary: -Thesis and DisSer


## Faz a tradução do resumo em inglês para português

In [43]:
my_final_summary_in_english = my_summary_in_english[0]['summary_text']
print(len(my_final_summary_in_english))

389


In [38]:
tokenizer_en_pt = AutoTokenizer.from_pretrained("unicamp-dl/translation-en-pt-t5")

tokenizer_config.json:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/756k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [39]:
model_en_pt = AutoModelForSeq2SeqLM.from_pretrained("unicamp-dl/translation-en-pt-t5")

config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [44]:
enpt_pipeline = pipeline('text2text-generation', model=model_en_pt, tokenizer=tokenizer_en_pt)
meu_resumo = enpt_pipeline(f"translate English to Portuguese: {my_final_summary_in_english}", max_length=len(my_final_summary_in_english))

In [46]:
if len(meu_resumo) > 0:
  print(meu_resumo[0]['generated_text'])

DESENVOLVIMENTO DA UNIVERSÍDIA FEDERAL DO PIAU Prezado Chefe de Sector, Eu Armando Sousa, UFPI Registro 1886865, localizado no Departamento de Informática, Centro de Ciência da Natureza, Universidade Federal do Ceará . Este certificado tem por objetivo cumprir as dificuldades provocadas pela pandemia do vírus Corona nos anos de 2020 e 2021 . Tese ou Projeto de Dissertação resumo: -Tese e Dissertação
